## Indice
#### 1. Pulizia crawled data
#### 2.Creo una colonna con i tag puliti
#### 3.Creazione network di partenza
#### 4.Creazione dataset da 10k nodi
###### 4.1 Controllo preliminare dei nodi ed edges
#### 5. Risultati

In [14]:
import twint
import pandas as pd


import networkx as nx
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import powerlaw
warnings.filterwarnings('ignore')


%matplotlib inline 

### PULIZIA CRAWLED DATA

In [15]:
df = pd.read_csv("climatechange.csv", sep='\t')

In [3]:
type(df)

pandas.core.frame.DataFrame

In [4]:
df.tail(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
50017,1175070693391765507,1175070693391765507,2019-09-20 17:34:19 ora legale Europa occidentale,2019-09-20,17:34:19,200,19811643,lvz,LVZ,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
50018,1175070453859094528,1175070453859094528,2019-09-20 17:33:22 ora legale Europa occidentale,2019-09-20,17:33:22,200,881570054801944576,gswnature,Georgie Ward,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [5]:
# Get all names 
for col_name in df.columns: 
    print(col_name)

id
conversation_id
created_at
date
time
timezone
user_id
username
name
place
tweet
language
mentions
urls
photos
replies_count
retweets_count
likes_count
hashtags
cashtags
link
retweet
quote_url
video
thumbnail
near
geo
source
user_rt_id
user_rt
retweet_id
reply_to
retweet_date
translate
trans_src
trans_dest


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50019 entries, 0 to 50018
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               50019 non-null  int64  
 1   conversation_id  50019 non-null  int64  
 2   created_at       50019 non-null  object 
 3   date             50019 non-null  object 
 4   time             50019 non-null  object 
 5   timezone         50019 non-null  int64  
 6   user_id          50019 non-null  int64  
 7   username         50019 non-null  object 
 8   name             50007 non-null  object 
 9   place            18 non-null     object 
 10  tweet            50019 non-null  object 
 11  language         50019 non-null  object 
 12  mentions         50019 non-null  object 
 13  urls             50019 non-null  object 
 14  photos           50019 non-null  object 
 15  replies_count    50019 non-null  int64  
 16  retweets_count   50019 non-null  int64  
 17  likes_count 

Toglie created_at, time, timezone, lasciando la data
Sono dati ripetuti 

In [16]:
df.drop(['trans_dest', 'trans_src','translate','retweet_date','geo','source','user_rt_id', 'user_rt','retweet_id',
        'near','place','thumbnail', 'time', 'timezone', 'video', 'photos', 'cashtags', 'created_at','quote_url','id','conversation_id'], axis=1, inplace=True)





In [5]:
#No valori nulli!
df['language'].isnull().values.any()

False

In [4]:
Lingue=list(set(df['language']))
#Lingue

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50019 entries, 0 to 50018
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               50019 non-null  int64 
 1   conversation_id  50019 non-null  int64 
 2   date             50019 non-null  object
 3   user_id          50019 non-null  int64 
 4   username         50019 non-null  object
 5   name             50007 non-null  object
 6   tweet            50019 non-null  object
 7   language         50019 non-null  object
 8   mentions         50019 non-null  object
 9   urls             50019 non-null  object
 10  replies_count    50019 non-null  int64 
 11  retweets_count   50019 non-null  int64 
 12  likes_count      50019 non-null  int64 
 13  hashtags         50019 non-null  object
 14  link             50019 non-null  object
 15  retweet          50019 non-null  bool  
 16  reply_to         50019 non-null  object
dtypes: bool(1), int64(6), object(10

Restingendo il range di tweet con numero di likes, retweet e replies
Si arriva a 34848 valori totali

In [17]:
df=df[df['likes_count']>=5]
df=df[df['replies_count']>=1]
df=df[df['retweets_count']>=3]

In [5]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34848 entries, 5 to 50018
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            34848 non-null  object
 1   user_id         34848 non-null  int64 
 2   username        34848 non-null  object
 3   name            34838 non-null  object
 4   tweet           34848 non-null  object
 5   language        34848 non-null  object
 6   mentions        34848 non-null  object
 7   urls            34848 non-null  object
 8   replies_count   34848 non-null  int64 
 9   retweets_count  34848 non-null  int64 
 10  likes_count     34848 non-null  int64 
 11  hashtags        34848 non-null  object
 12  link            34848 non-null  object
 13  retweet         34848 non-null  bool  
 14  reply_to        34848 non-null  object
dtypes: bool(1), int64(4), object(10)
memory usage: 4.0+ MB


### Creo una colonna con i tag puliti

In [18]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mir-9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Faccio questa funzione per pulire i tags dalle quadre, la applico a una nuova colonna (clean_tags)

In [23]:
def remove_content(text):
    text = re.sub(r"\[", "", text) #remove ]
    text=re.sub(r'\]','',text) #remove ]
    text=re.sub(r'\'','',text) #remove apici
    text=re.sub(r'\n','',text) #remove spazio vuoto

    return text

In [24]:
len(df['hashtags'])

34848

In [33]:
df['clean_tags']=df['hashtags'].apply(lambda x: remove_content(x))
len(df['clean_tags'])

34848

In [26]:
df.insert(0, 'clean_tags', df.pop('clean_tags'))

In [27]:
df.insert(1, 'hashtags', df.pop('hashtags'))

In [29]:
df.head(1)

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,urls,replies_count,retweets_count,likes_count,link,retweet,reply_to
5,"fossilfuels, wind, solar, green, hydrogen, ene...","['fossilfuels', 'wind', 'solar', 'green', 'hyd...",2022-07-10,59386332,arikring,Arik Ring - Energy Engineering Expert,@ProfStrachan @ECOWARRIORSS @ChristineMilne @m...,en,"[{'screen_name': 'guytaur', 'name': 'scott. tr...",['https://reneweconomy.com.au/integrated-wind-...,2,54,59,https://twitter.com/arikring/status/1546251358...,False,"[{'screen_name': 'ProfStrachan', 'name': 'Prof..."


In [39]:
#Salvo le modifiche nel df nuovo
df.to_csv('fff.csv', index=False)

###  Creazione network di partenza

In [40]:
df1=pd.read_csv("fff.csv")
df1.head(1)

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,urls,replies_count,retweets_count,likes_count,link,retweet,reply_to
0,"fossilfuels, wind, solar, green, hydrogen, ene...","['fossilfuels', 'wind', 'solar', 'green', 'hyd...",2022-07-10,59386332,arikring,Arik Ring - Energy Engineering Expert,@ProfStrachan @ECOWARRIORSS @ChristineMilne @m...,en,"[{'screen_name': 'guytaur', 'name': 'scott. tr...",['https://reneweconomy.com.au/integrated-wind-...,2,54,59,https://twitter.com/arikring/status/1546251358...,False,"[{'screen_name': 'ProfStrachan', 'name': 'Prof..."


In [42]:
#Ti crea un dizionario in cui le chiavi sono i tag singoli e i valori sono gli id degli user
tags_dict = {}
for i in range(len(df1)):
    ll = str(df1["clean_tags"][i]).split(",")[:-1]
    for tg in ll:
        if tg not in tags_dict:
            tags_dict[tg] = [df1["user_id"][i]]
        else:
            tags_dict[tg].append(df1["user_id"][i])

In [43]:
#Crea la lista di tags dalle chiavi del dizionario e li associa alla variabile tag
tags = list(tags_dict.keys())
#crea una lista costituita da coppie di tags
couple_tags = []
for i in range(len(tags)):
    for j in tags[i+1:]:
        couple_tags.append((tags[i], j)) 

In [47]:
couple_tags

[('fossilfuels', ' wind'),
 ('fossilfuels', ' solar'),
 ('fossilfuels', ' green'),
 ('fossilfuels', ' hydrogen'),
 ('fossilfuels', ' energy'),
 ('fossilfuels', ' climateaction'),
 ('fossilfuels', ' renewableenergy'),
 ('fossilfuels', ' carbon'),
 ('fossilfuels', ' demvoice1'),
 ('fossilfuels', ' sdgs'),
 ('fossilfuels', ' climatecrisis'),
 ('fossilfuels', 'klimakrise'),
 ('fossilfuels', ' govegan'),
 ('fossilfuels', 'fridaysforfuture'),
 ('fossilfuels', ' riseupmovement'),
 ('fossilfuels', 'notmytaxonomy'),
 ('fossilfuels', 'gocarfree'),
 ('fossilfuels', ' istayontheground'),
 ('fossilfuels', ' flightfree'),
 ('fossilfuels', ' eatonlyplants'),
 ('fossilfuels', ' shopstop'),
 ('fossilfuels', ' fridaysforfuture'),
 ('fossilfuels', 'climatecrisis'),
 ('fossilfuels', ' climatestrike'),
 ('fossilfuels', ' climate'),
 ('fossilfuels', ' savesoilbengaluru'),
 ('fossilfuels', ' gaskrise'),
 ('fossilfuels', ' grünen'),
 ('fossilfuels', 'peoplepower'),
 ('fossilfuels', ' actnow'),
 ('fossilfuels'

In [44]:
len(couple_tags)

97489666

In [45]:
len(tags)

13964

In [35]:
len(tags)

13964

In [46]:
source = []
target = []
count = []
nodes = []
for i in couple_tags:
    ll = list(set(tags_dict[i[0]]) & set(tags_dict[i[1]]))
    if len(ll)>0:
        source.append(i[0])
        target.append(i[1])
        count.append(len(ll))
        nodes.append(i[0])
        nodes.append(i[1])

In [48]:
nodes = list(set(nodes)) #list()crea una lista; set(nodes) prende i nodi senza ripetizioni

In [49]:
len(nodes)

13703

In [50]:
nodes

['dannenröderforst',
 'gutenmorgen',
 ' nazis',
 ' zensur',
 'eyesopencanada',
 ' supremecourt',
 ' factoryfarming',
 ' contacttracing',
 'tigers',
 ' weekendvibes',
 ' btw21',
 ' climatecrisisbangladesh',
 ' sdgs',
 'rightlivelihoodaward',
 ' shopstop',
 ' unprecedentedchangenow',
 ' streatweare',
 ' saveforest',
 ' neversurrender',
 ' refugisclimàtics',
 ' airbag',
 ' carbonrationtickets',
 ' sars_cov2',
 ' napoli',
 ' notredame',
 ' revoltaescolar',
 'fornature',
 ' marburg',
 ' forzamafia',
 ' yoyoma',
 '28settembre',
 'gewalt',
 ' southpole',
 ' bolloré',
 ' biodiversityday',
 'hambibleibt',
 'rivers',
 ' economiacivile',
 ' frackedgas',
 'deepclimatestrike',
 'australiafires',
 'veganism',
 ' schoolstrike4cliamte',
 ' zeitgemäß',
 ' klimawahnsinn',
 ' covid19',
 ' viaggio',
 ' wastefree',
 'planetoverprofit',
 ' just',
 'raceformars',
 ' faceclimateemergeny',
 ' friedenfürsklima',
 ' djvielfalt',
 ' konzert',
 ' mutlangen',
 ' savebuxwahasavelife',
 ' fail',
 'bildungabersicher',

In [51]:
new_df = pd.DataFrame({"Id": nodes})
new_df.to_csv("../Fridayforfuture/nodes.csv", index=False)

In [52]:
new_df = pd.DataFrame({"Source": source,
                       "Target": target,
                      "Count": count})
new_df.to_csv("../Fridayforfuture/edges.csv", index=False)

### Creazione dataset da 10k nodi

###### Controllo preliminare dei nodi ed edges


In [53]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import math
import numpy as np
import networkx as nx
from operator import itemgetter

In [54]:
path = "../Fridayforfuture/edges.csv"
path_all = "../Fridayforfuture/fff.csv"
path_nodes = "../Fridayforfuture/nodes.csv"

In [55]:
fff = pd.read_csv(path)

In [56]:
fff.sort_values('Count', ascending=False)

,Source,Target,Count
12902,fridaysforfuture,fridaysforfuture,842
20613,fridaysforfuture,climatestrike,650
20648,fridaysforfuture,climatestrike,472
12904,fridaysforfuture,climatestrike,439
2093,climateaction,fridaysforfuture,395
...,...,...,...
156606,langstreckenluisa,gruenejugend,1
156605,langstreckenluisa,flutkatastrophe,1
156604,langstreckenluisa,umweltkatastrophe,1
156603,langstreckenluisa,stockolores,1


In [57]:
G = nx.read_edgelist(path, comments = "S", delimiter = ",", data=(("Count", int),))

In [58]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')
sorted_degree = sorted(degree_dict.items(), key = itemgetter(1), reverse = True)
print(" ---- Top 20 nodes by degree ----")
for d in sorted_degree[:20]:
    print("\t" + str(d))

 ---- Top 20 nodes by degree ----
	('fridaysforfuture', 11712)
	('climatestrike', 5194)
	('climateaction', 3894)
	('climatecrisis', 3804)
	('climateemergency', 3379)
	('climatechange', 2914)
	('climatejustice', 2856)
	('climatestrikeonline', 2625)
	('klimastreik', 2622)
	('fridayforfuture', 2569)
	('klimaschutz', 2530)
	('klimakrise', 2371)
	('gretathunberg', 2284)
	('climateactionnow', 2239)
	('peoplenotprofit', 2160)
	('nomoreemptypromises', 2104)
	('fff', 2091)
	('uprootthesystem', 1991)
	('climate', 1986)
	('extinctionrebellion', 1954)


In [59]:
nodes = []
others = []
for i in range(len(sorted_degree)):
    if i<100002:
        nodes.append(sorted_degree[i][0])
        #print("N.nodi:",nodes)
    else:
        others.append(sorted_degree[i][0])

##### Abbiamo 18k nodi

In [60]:
len(nodes)

18887

In [61]:
new_fff = fff[(fff["Source"].isin(nodes)) & (fff["Target"].isin(nodes))] 

In [62]:
new_df.to_csv("../Fridayforfuture/edges_10K.csv",  index=False)

In [63]:
new_df = pd.DataFrame({"Id": nodes})
new_df.to_csv("../Fridayforfuture/nodes_10K.csv", index=False)

In [64]:
other_df = pd.DataFrame({"Id": others})
other_df.to_csv("../Fridayforfuture/nodes_3K.csv", index=False)

In [43]:
edges=pd.read_csv('edges_10k.csv')

## Risultato pulizia:

In [65]:
print(nx.info(G))

Graph with 18887 nodes and 363452 edges
